In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp

from math import*

Potential

In [2]:
def V_harmonic(phi,m2):
    return(0.5*m2*phi*phi)

def dV_harmonic(phi,m2):
    return(m2*phi)

def ddV_harmonic(phi,m2):
    return(m2)

In [3]:
from math import pi
csteGrav = 6.6743015e-11
M_pl = 1/np.sqrt(8*pi*csteGrav)
# Units #Internally, the code uses almost everywhere units of Mpc to some power, excepted in the inflation module, where many quantities are in natural units (wrt the true Planck mass).


def V_brane(phi,M4,mu,n):
    return(M4/(1+np.power(mu/phi,n)))

def V_brane_shift(phi,phi0,M4,mu,n):
    return(V_brane(phi0-phi,M4,mu,n))

def dV_brane(phi,M4,mu,n):
    factor = M4*n*np.power(mu,n)
    denom = np.power(phi,n)+np.power(mu,n)
    return(factor*np.power(phi,n-1)/np.power(denom,2))

def dV_brane_shift(phi,phi0,M4,mu,n):
    return(-dV_brane(phi0-phi,M4,mu,n))

def ddV_brane(phi,M4,mu,n):
    factor = M4*n*pow(mu,n)
    factor2 = (n-1)*np.power(mu,n)-(n+1)*np.power(phi,n)
    denom = np.power(mu,n)+np.power(phi,n)
    return(factor*factor2/np.power(denom,3))

def ddV_brane_shift(phi,phi0,M4,mu,n):
    return(ddV_brane(phi0-phi,M4,mu,n))

In [4]:
def epsilon1_true(phi,M4,mu,n):
    factor = 0.5*pow(M_pl,2)
    return(factor*pow(dV_brane(phi,M4,mu,n)/V_brane(phi,M4,mu,n),2)  )

def epsilon1_true_shift(phi,phi0,M4,mu,n):
    return(epsilon1_true(phi0-phi,M4,mu,n))

Friedmann Equation and Klein-Gordon equation

In [5]:
def equations_scalar_inflation_BI(t, y,M4,mu,n):

    def V(phi):
        return(V_brane(phi,M4,mu,n))

    phi, phi_dot = y

        # Friedmann equation :
    H = 1/M_pl*np.sqrt( (1/3) * (0.5 * phi_dot**2 + V(phi)) )
    
    dV = dV_brane(phi,M4,mu,n)  
    
    # Klein-Gordon equation
    phi_dot_dot = -3 * H * phi_dot - dV 
    
    return [phi_dot, phi_dot_dot]


In [6]:
def equations3_scalar_inflation_BI(t, y,M4,mu,n):

    def V(phi):
        return(V_brane(phi,M4,mu,n))

    phi, phi_dot, a = y

        # Friedmann equation :
    H = 1/M_pl*np.sqrt( (1/3) * (0.5 * phi_dot**2 + V(phi)) )
    
    dV = dV_brane(phi,M4,mu,n)  
    
    # Klein-Gordon equation
    phi_dot_dot = -3 * H * phi_dot - dV 
    
    a_dot = a*H

    return [phi_dot, phi_dot_dot, a_dot]


Friedmann and Klein-Gordon equations with a Shifted potential (decreasing and >0)

In [7]:
def equations_scalar_inflation_shifted_BI(t, y,phi_star,M4,mu,n):

    def V(phi):
        return(V_brane_shift(phi,phi_star,M4,mu,n))

    phi, phi_dot, a = y

        # Friedmann equation :
    H = 1/M_pl*np.sqrt( (1/3) * (0.5 * phi_dot**2 + V(phi)) )
    
    dV = dV_brane_shift(phi,phi_star,M4,mu,n)  
    
    # Klein-Gordon equation
    phi_dot_dot = -3 * H * phi_dot - dV 
    
    a_dot = a*H

    return [phi_dot, phi_dot_dot, a_dot]


Calculation of $\phi_{end}$  (condition : $\epsilon_V = 1$)

In [8]:
def calculate_phi_end(M4,mu,n):
    
    a=-np.sqrt(0.5)*n*pow(mu,n)
    b=pow(mu,4)

    roots = np.roots([1,0,0,0,b,a])
    positive_real_roots = [root for root in roots if np.isreal(root) and root > 0]

    if len(positive_real_roots) > 0:
        return positive_real_roots[0].real 
    else:
        print("No positive real root found.")
        return None

Calculate  $\phi_{start}$ in a bad way, but just for test

In [9]:
## précision nécessaire pour pouvoir atteindre phi tel que  V(phi) proche de M4   (on utilise cette précision dans la fonction calculate phi start)
## Attention, si et seulement si   M_pl = 1
def precision_mu(mu):
    return(pow(10,(-2+4*np.log10(mu/M_pl))))

#Calcule phi tel que V(phi) proche de M4 ... la précision recquise est expliquée après  (en fait elle dépend de M4 et donc de mu  (car M4/mu4 = 16/27))
def calculate_phi_start_BI(M4,mu,n,precision):
    return(mu*pow(M4/precision,1/n))

Initial condition $\frac{d\phi}{dt} \big ( t=t_* \big ) = ... = $ slow roll approx $=$

In [10]:
def phi_dot_star_BI(phi_star,M4,mu,n):
    factor = -sqrt(3)*M_pl
    return(factor*dV_brane(phi_star,M4,mu,n)/sqrt(V_brane(phi_star,M4,mu,n)))

In [11]:
def phi_dot_star_shifted_BI(phi_star,M4,mu,n):
    factor = -sqrt(3)*M_pl
    return(factor*dV_brane(phi_star,M4,mu,n)/sqrt(V_brane(phi_star,M4,mu,n)))

Résolution ED en prenant shifted potential:

- on donne une valeur de $\phi_*$, on intègre l'ED pour trouver $\phi$ et $a$
- on calcule la valeur d' $e$ -folds associée
- on recommence l'opération jusqu'à avoir $60 e$ -folds

Résolution ED

In [12]:
def resolution_scalar_inflation_BI(phi_star,M4,mu,n):
    phi_dot_star = phi_dot_star_BI(phi_star,M4,mu,n)


    y0 = [phi_star,phi_dot_star]

    t_span = (0,1)

    def equations(t,y):
        return(equations_scalar_inflation_BI(t,y,M4,mu,n))

    solution = solve_ivp(equations,t_span,y0,method='RK45')

    t = solution.t
    phi = solution.y[0]
    phi_dot = solution.y[1]

    return(t,phi,phi_dot)


Idem, mais avec $\phi,\dot{\phi}, a$

In [13]:
def resolution3_scalar_inflation_BI(phi_star,M4,mu,n):
    phi_dot_star = phi_dot_star_BI(phi_star,M4,mu,n)
    a_star = 1

    y0 = [phi_star,phi_dot_star,a_star]

    t_span = (0,1)  #combien?

    def equations3(t,y):
        return(equations3_scalar_inflation_BI(t,y,M4,mu,n))

    solution = solve_ivp(equations3,t_span,y0,method='RK45')

    t = solution.t
    phi = solution.y[0]
    phi_dot = solution.y[1]
    a = solution.y[0]

    return(t,phi,phi_dot,a)


Résolution ED et plot

In [14]:
def couleur(i,N):
    myColor = plt.get_cmap("Dark2")
    return(myColor((i+1)/N))

In [22]:
def resolution_BI_plot(phi_star,M4,mu,n,c,c2):
    t,phi,phi_dot = resolution_scalar_inflation_BI(phi_star,M4,mu,n)
    t2,phi2,phi_dot2,a2 = resolution3_scalar_inflation_BI(phi_star,M4,mu,n)

    fig = plt.figure(figsize=(10,7))
    ax1 = plt.subplot(2,2,1)
    ax2 = plt.subplot(2,2,2)
    ax3 = plt.subplot(2,2,3)
    ax4 = plt.subplot(2,2,4)

    V = V_brane(phi,M4,mu,n)
    V2 = V_brane(phi2,M4,mu,n)
    
    dV = dV_brane(phi,M4,mu,n)


    ax1.plot(t,phi,color=c,label="2")
    ax1.set_ylabel(r'$\phi(t)$')
    ax1.set_xlabel(r'$t$')

    ax2.plot(t,phi_dot,color=c,label="2")
    ax2.set_ylabel(r'$\frac{d\phi}{dt}$')
    ax2.set_xlabel(r'$t$')

    ax3.plot(phi,V,color=c,label="2")
    ax3.set_ylabel(r'$V(\phi)$')
    ax3.set_xlabel(r'$\phi$')

    print("temps",t)
    ax1.legend()
    ax2.legend()
    ax3.legend()
    ax4.legend()

    plt.show()

    fig = plt.figure(figsize=(10,7))
    ax1 = plt.subplot(2,2,1)
    ax2 = plt.subplot(2,2,2)
    ax3 = plt.subplot(2,2,3)
    ax4 = plt.subplot(2,2,4)
    ax1.plot(t2,phi2,color=c2,label="3")
    ax1.set_ylabel(r'$\phi(t)$')
    ax1.set_xlabel(r'$t$')

    ax2.plot(t2,phi_dot2,color=c2,label="3")
    ax2.set_ylabel(r'$\frac{d\phi}{dt}$')
    ax2.set_xlabel(r'$t$')

    ax3.plot(phi,V,color=c2,label="3")
    ax3.set_ylabel(r'$V(\phi)$')
    ax3.set_xlabel(r'$\phi$')

    ax4.plot(t2,a2,color=c2,label="3")
    ax4.set_ylabel(r'$a(t)$')
    ax4.set_xlabel(r'$t$')

    print("temps 2",t2)
    ax1.legend()
    ax2.legend()
    ax3.legend()
    ax4.legend()

    plt.show()
    

In [29]:
def essai_resolution_BI_plot():

    n=4
    liste_mu = M_pl*np.logspace(0,0,1)
    liste_M4=4*np.pi*np.pi*np.power(liste_mu,4)*16/27
    
    liste_eta = [ precision_mu(mu) for mu in liste_mu ]

    
    NN=len(liste_mu)
    for i in range(NN):
        mu = liste_mu[i]
        M4=liste_M4[i]
        eta=liste_eta[i]
        phi_star  = calculate_phi_start_BI(M4,mu,n,eta)

        print(phi_star)
        phi_star = M_pl*pow(10,20)
        print(phi_star)
        print("")
        resolution_BI_plot(phi_star,M4,mu,n,"b","r")


essai_resolution_BI_plot()

4146038282.6402826
2.441614860380852e-96



/tmp/ipykernel_318/536833619.py:8: RuntimeWarning: overflow encountered in power
  return(M4/(1+np.power(mu/phi,n)))
/tmp/ipykernel_318/1183339483.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  return(factor*dV_brane(phi_star,M4,mu,n)/sqrt(V_brane(phi_star,M4,mu,n)))


Cacul de $\Delta N = \int_{a_{end}}^{a_*} dlna = \int_{\phi_{end}}^{\phi_*} \frac{H(\phi)}{\dot{\phi}} d\phi$

In [17]:
from scipy import integrate



def calcule_N(phi1,phi2,M4,mu,n):
    def f(phi):
        return()
    N = integrate.romberg(f,phi1,phi2)
    return(N)

In [18]:
def resolution_and_calculate_DN_BI(phi_star,M4,mu,n):
    t,phi,phi_dot = resolution_scalar_inflation_BI(phi_star,M4,mu,n)

    

    DN=0
    DN = 1
